# Project 3 - Water Pump Clasificaiton

# Notes

**Things to add to my project - ideas from class**

Anything?


# Setup

In [1]:
from __future__ import print_function

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import division
pd.set_option('display.width',5000)

In [3]:
import patsy

from sklearn import linear_model as lm
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import confusion_matrix

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data Import - Training Data

This data is part of the Data Driven Competition

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/


In [4]:
# 'Values' provided for each pump location - features

df_values = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/Training set values.csv')
df_values.sample(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
49396,46465,0.0,2012-10-21,Rwssp,0,DWE,32.316785,-4.031537,Butibu A,0,...,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,other,other
18003,57882,200.0,2011-03-23,Is,275,Is,37.035127,-7.692044,Kwa Kalua,0,...,on failure,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
49975,60360,0.0,2012-10-30,Grumeti,1330,GRUMETI,34.465963,-2.047859,Shule Ya Msingi Makundusi,0,...,never pay,salty,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe


In [5]:
# 'Labels' provided for each pump location - clasificaiton - what I am predicting

df_labels = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/Training set labels.csv')
df_labels.sample(3)

,id,status_group
49038,42229,functional
53877,9616,non functional
21883,68361,functional


# Data Import - Challenge Data

In [35]:
# 'Values' provided for each competition pump location - features

df_test_values = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/Test set values.csv')
df_test_values.sample(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
2225,33644,20.0,2011-03-02,Government Of Tanzania,45,DWE,38.575684,-6.801949,Enzimbali,0,...,per bucket,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
6683,43537,0.0,2013-03-15,Vwc,1337,VWC,36.754739,-2.590400,Kwa Alaatati,0,...,on failure,soft,good,insufficient,insufficient,river,river/lake,surface,cattle trough,cattle trough
6506,11725,50.0,2013-01-26,Jica,273,Kokeni,38.841901,-10.150719,Namchachamba,0,...,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump


In [36]:
# 'Labels' I will provide for each pump location - clasificaiton

df_sub = pd.read_csv('/Users/amycurneen/ds/metis/metisgh/Metis-Curneen/3 - Water Pumps/Data Downloads/SubmissionFormat.csv')
df_sub.sample(3)

,id,status_group
169,16862,predicted label
8430,45277,predicted label
7662,70655,predicted label


In [37]:
df_sub = df_sub.drop('status_group', axis = 1)

# Exploratory Data Analysis

## Colum descriptions

My goal is to predict the operating condition of a waterpoint for each record in the dataset. I was provided the following set of information about the waterpoints:

* amount_tsh - Total static head (amount water available to waterpoint)
    * 98 unique
* date_recorded - The date the row was entered
    * 365 unique
    * year - month - day
* funder - Who funded the well
    * 1897 unique
    * look at top ones?
* installer - Organization that installed the well
    * 2145 unique
    * DWE is main one - 10x closest other, 17k
* wpt_name - Name of the waterpoint if there is one
    * 37400 unique
    * look at top ones?
* num_private - (NO PROVIDED DESC)
    * 65 unique
    * USELESS FEATURE
* population - Population around the well
    * 1049 unique
    * a lot are zero
* public_meeting - True/False
    * 2 unique
* recorded_by - Group entering this row of data
    * 1 unique
    * all the same - USELESS FEATURE
* scheme_management - Who operates the waterpoint
    * 12 unique
* scheme_name - Who operates the waterpoint
    * 2696 unique
    * USELESS FEATURE
* permit - If the waterpoint is permitted
    * 2 unique
* construction_year - Year the waterpoint was constructed
    * 55 unique
    * third are 0 - USELESS FEATURE


* Geography
    * gps_height - Altitude of the well
        * numerical
    * longitude - GPS coordinate
        * numerical
    * latitude - GPS coordinate
        * numerical
    * basin - Geographic water basin
        * 9 unique
    * subvillage - Geographic location
        * 19287 unique
    * region - Geographic location
        * 21 unique
    * region_code - Geographic location (coded)
        * 27 unique
    * district_code - Geographic location (coded)
        * 20 unique
    * lga - Geographic location
        * 125 unique
    * ward - Geographic location
        * 2092 unique


* Extraction
    * extraction_type - The kind of extraction the waterpoint uses
        * 18 unique
        * Most descriptive of extraction
    * extraction_type_group - The kind of extraction the waterpoint uses
        * 13 unique
        * Parent of extraction_type
    * extraction_type_class - The kind of extraction the waterpoint uses
        * 7 unique
        * Parent of extraction_type_group


* Overhead
    * management - How the waterpoint is managed
        * 12 unique
    * management_group - How the waterpoint is managed
        * 5 unique
    * payment - What the water costs
        * 7 unique
        * same as payment type
    * payment_type - What the water costs
        * 7 unique
        * same as payment


* Water
    * water_quality - The quality of the water 
        * 3 unique
        * Subset of quality_group
    * quality_group - The quality of the water
        * 6 unique
        * Parent group of water_quality
    * quantity - The quantity of water
        * 5 unique
        * Same as quantity_group
    * quantity_group - The quantity of water
        * 5 unique
        * Same as quantity
    * source - The source of the water
        * 10 unique
    * source_type - The source of the water
        * 7 unique
        * Subset of source
    * source_class - The source of the water
        * 3 unique
        * Subset of source_type
    * waterpoint_type - The kind of waterpoint
        * 6 unique
        * Parent of waterpoint_type_group
    * waterpoint_type_group - The kind of waterpoint
        * 7 unique
        * Subset of waterpoint_type

## Various EDA

##### Status

In [ ]:
# 3 Unique status present
df_labels.status_group.value_counts()

##### Review histograms of extraction data

* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses

In [ ]:
df_values.extraction_type.value_counts().plot('barh')
len(df_values.extraction_type.value_counts())

In [ ]:
df_values.extraction_type_group.value_counts().plot('barh')
len(df_values.extraction_type_group.value_counts())

In [ ]:
df_values.extraction_type_class.value_counts().plot('barh')
len(df_values.extraction_type_class.value_counts())

##### Other

In [ ]:
# df_values.population.value_counts()

In [8]:
len(df_values.ward.value_counts())

2092

In [ ]:
# len(df_values.management.value_counts())

In [ ]:
# print(df_values.region.value_counts())
# print('Length: ',len(df_values.region.value_counts()))

In [ ]:
#print(df_values.region_code.value_counts())
#print('Length: ',len(df_values.region_code.value_counts()))

In [ ]:
# df_values.water_quality.value_counts()

In [ ]:
#df_values.installer.value_counts()

# Test Train Split

In [9]:
# randomize data
# df_lables, df_values - combine and shuffle this data

df = pd.merge(df_labels,df_values,how = 'left')
df = df.sample(frac=1).reset_index(drop=True)

In [10]:
# Split back to X and y and then TTS
y = df[["id","status_group"]]
X = df.drop('status_group', axis=1)

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0)

In [13]:
X_train.shape

(59400, 40)

# Decision Tree

## Create DT on test-train-split

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
df.sample(3)

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
21789,71926,non functional,0.0,2011-02-28,Government Of Tanzania,18,Centr,38.236501,-7.773575,Mwenyekiti,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
56815,3571,functional needs repair,0.0,2011-04-08,Dwe,0,DWE,33.100161,-8.967003,Msukwa,...,on failure,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
26898,4044,functional,4000.0,2011-02-20,Cefa,1963,CEFA,35.884190,-8.226280,none,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe


In [16]:
# try with a few less features first 

feature_no_sel = df[['amount_tsh', 'date_recorded', 'funder', 'installer', 
                   'wpt_name', 'num_private', 'subvillage', 'region', 
                   'region_code', 'district_code', 'lga', 'ward', 'public_meeting', 'recorded_by', 
                   'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 
                   'extraction_type_group', 'management', 'management_group', 'payment', 
                   'payment_type', 'water_quality', 'quality_group', 'quantity_group', 'source', 
                   'source_type', 'waterpoint_type','region_code', 
                  'district_code','public_meeting','scheme_management']]

feature_sel = df[['id', 'status_group', 'gps_height', 'longitude', 'latitude', 'basin',
                  'extraction_type_class','payment','quantity','water_quality', 'source_class',
                  'waterpoint_type_group','population','public_meeting','scheme_management']]

In [17]:
feature_sel.sample(3)

,id,status_group,gps_height,longitude,latitude,basin,extraction_type_class,payment,quantity,water_quality,source_class,waterpoint_type_group,population,public_meeting,scheme_management
20704,12335,functional,0,31.541619,-1.672945,Lake Victoria,gravity,never pay,enough,soft,groundwater,communal standpipe,0,True,VWC
26593,17633,functional,1283,35.184865,-8.738531,Rufiji,handpump,never pay,enough,soft,groundwater,hand pump,0,True,VWC
42366,14705,functional,984,35.484878,-10.628221,Lake Nyasa,gravity,pay when scheme fails,enough,soft,surface,communal standpipe,0,True,VWC


In [18]:
# Change what I predict to 1,2,3

for i in range(len(feature_sel.status_group)):
    if feature_sel.status_group[i] == 'functional':
        feature_sel.at[i, 'status_group'] = 3
    elif feature_sel.status_group[i] == 'functional needs repair':
        feature_sel.at[i, 'status_group'] = 2
    else:
        feature_sel.at[i, 'status_group'] = 1

In [19]:
feature_sel=feature_sel.drop('id', axis=1)
feature_sel.sample(3)

,status_group,gps_height,longitude,latitude,basin,extraction_type_class,payment,quantity,water_quality,source_class,waterpoint_type_group,population,public_meeting,scheme_management
40435,3,1578,34.456198,-9.064051,Rufiji,gravity,pay when scheme fails,enough,soft,surface,communal standpipe,56,True,WUA
21286,3,206,39.078220,-9.862540,Ruvuma / Southern Coast,handpump,never pay,enough,salty,groundwater,hand pump,1680,True,VWC
38402,2,281,39.011854,-10.487682,Ruvuma / Southern Coast,gravity,never pay,insufficient,soft,groundwater,communal standpipe,49,True,VWC


In [20]:
# rearange columns
cols = feature_sel.columns.tolist()
cols = ['status_group',
             'gps_height',
             'longitude',
             'latitude',
             'population',
             'basin',
             'extraction_type_class',
             'payment',
             'quantity',
             'water_quality',
             'source_class',
             'waterpoint_type_group',
             'public_meeting',
             'scheme_management']
feature_sel = feature_sel[cols]
feature_sel.sample(3)

,status_group,gps_height,longitude,latitude,population,basin,extraction_type_class,payment,quantity,water_quality,source_class,waterpoint_type_group,public_meeting,scheme_management
18146,3,1403,34.686050,-2.146819,140,Lake Victoria,motorpump,never pay,insufficient,salty,groundwater,communal standpipe,NaN,WUG
48082,1,1313,30.524735,-3.798826,230,Lake Tanganyika,gravity,unknown,enough,soft,surface,communal standpipe,True,VWC
39670,3,1290,36.893674,-3.108039,350,Internal,gravity,unknown,insufficient,soft,groundwater,communal standpipe,True,VWC


In [21]:
#create dummy features for applicabe

feature_sel = pd.get_dummies(feature_sel,columns=list(feature_sel.columns[5:]))
feature_sel.sample(3)

,status_group,gps_height,longitude,latitude,population,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,...,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority
40688,3,0,0.000000,-2.000000e-08,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
28447,3,0,31.173026,-1.560288e+00,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
19920,2,983,29.724092,-4.786516e+00,400,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [22]:
columns = list(feature_sel.columns)
feature_sel[columns[5:]] = feature_sel[columns[5:]].astype(int)
feature_sel[columns[0]] = feature_sel[columns[0]].astype(int)

In [23]:
y = feature_sel.status_group
X = feature_sel.drop('status_group', axis=1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0)

In [25]:
decisiontree = DecisionTreeClassifier(random_state=0)
decisiontree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [26]:
decisiontree.score(X_test,y_test)

ValueError: Found array with 0 sample(s) (shape=(0, 63)) while a minimum of 1 is required.

## DT on competition datsa

In [38]:
df_test_values = df_test_values[['gps_height', 'longitude', 'latitude', 'population','basin',
                  'extraction_type_class','payment','quantity','water_quality', 'source_class',
                  'waterpoint_type_group','public_meeting','scheme_management']]

In [39]:
df_test_values

,gps_height,longitude,latitude,population,basin,extraction_type_class,payment,quantity,water_quality,source_class,waterpoint_type_group,public_meeting,scheme_management
0,1996,35.290799,-4.059696e+00,321,Internal,other,never pay,seasonal,soft,surface,other,True,Parastatal
1,1569,36.656709,-3.309214e+00,300,Pangani,gravity,never pay,insufficient,soft,groundwater,communal standpipe,True,VWC
2,1567,34.767863,-5.004344e+00,500,Internal,other,never pay,insufficient,soft,surface,other,True,VWC
3,267,38.058046,-9.418672e+00,250,Ruvuma / Southern Coast,other,unknown,dry,soft,groundwater,other,NaN,VWC
4,1260,35.006123,-1.095041e+01,60,Ruvuma / Southern Coast,gravity,pay monthly,enough,soft,groundwater,communal standpipe,NaN,Water Board
5,1685,36.685279,-3.302420e+00,200,Pangani,gravity,never pay,enough,soft,groundwater,communal standpipe,True,VWC
6,550,36.398041,-7.541382e+00,600,Rufiji,handpump,never pay,enough,salty,groundwater,hand pump,True,VWC
7,234,39.607420,-1.089379e+01,1,Ruvuma / Southern Coast,submersible,never pay,dry,soft,groundwater,communal standpipe,True,Water Board
8,584,39.262951,-1.082359e+01,40,Ruvuma / Southern Coast,gravity,pay per bucket,insufficient,soft,groundwater,communal standpipe,True,VWC
9,1083,37.096108,-3.251754e+00,1,Pangani,gravity,pay monthly,enough,soft,groundwater,communal standpipe,True,Water Board


In [29]:
# rearange columns
# cols = df_test_values.columns.tolist()
# cols = ['gps_height',
#              'longitude',
#              'latitude',
#              'population',
#              'basin',
#              'extraction_type_class',
#              'payment',
#              'quantity',
#              'water_quality',
#              'source_class',
#              'waterpoint_type_group',
#              'public_meeting',
#              'scheme_management']
# df_test_values = df_test_values[cols]

In [40]:
df_test_values = pd.get_dummies(df_test_values,columns=list(df_test_values.columns[4:]))
df_test_values.sample(3)

,gps_height,longitude,latitude,population,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,...,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority
3132,0,32.967188,-2.639319,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5890,1625,37.121072,-3.160426,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
7742,1800,34.663946,-1.362950,200,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
a = [0] * len(df_test_values['gps_height'])
df_test_values['scheme_management_None'] = a
df_test_values.sample(3)

,gps_height,longitude,latitude,population,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,...,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,scheme_management_None
1216,1246,30.136968,-4.540764,320,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
6218,0,32.809861,-4.180536,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2323,0,31.025836,-1.006316,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
columns = list(df_test_values.columns)
df_test_values[columns[4:]] = df_test_values[columns[4:]].astype(int)
df_test_values[columns[0]] = df_test_values[columns[0]].astype(int)

## Apply to competition data

In [ ]:
df_test_labels = decisiontree.predict(df_test_values)

In [ ]:
df_test_labels.shape

In [ ]:
df_sub['status_group'] = df_test_labels

In [ ]:
df_sub['status_group'] = df_sub['status_group'].astype(str)

for i in range(len(df_sub.status_group)):
    if df_sub.status_group[i] == '3':
        df_sub.at[i, 'status_group'] = 'functional'
    elif df_sub.status_group[i] == '2':
        df_sub.at[i, 'status_group'] = 'functional needs repair'
    else:
        df_sub.at[i, 'status_group'] = 'non functional'
df_sub.sample(3)

In [ ]:
df_sub.status_group.value_counts()

In [ ]:
df_sub.to_csv('./Submissions/decisionTree.csv',index=False)

## Ranking

<table>
  <tr>
    <th>Best</th>
    <th>Current Rank</th>
    <th>Competitors</th>
    <th>Time</th>
  </tr>
  <tr>
    <td>0.6449</td>
    <td>1309</td>
    <td>5059</td>
    <td>May 7, 2018, 11:50 p.m.</td>
  </tr>
</table>

## Adding depths to tree

In [35]:
from sklearn.cross_validation import cross_val_score

depth = []
for i in range(3,20):
    clf = DecisionTreeClassifier(max_depth=i)
    # Perform 7-fold cross validation 
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=7, n_jobs=4)
    depth.append((i,scores.mean()))
print(depth)

[(3, 0.6933672940058565), (4, 0.7014481277774461), (5, 0.7074076640671273), (6, 0.7138723251449507), (7, 0.7196972711208949), (8, 0.7248148250084826), (9, 0.7298822743078776), (10, 0.7353201608784905), (11, 0.7413808625996199), (12, 0.7455050707863776), (13, 0.7500506362634685), (14, 0.7530640564635427), (15, 0.7560100336891841), (16, 0.7591920404217628), (17, 0.76151517870262), (18, 0.7620879944308843), (19, 0.7628117864963532)]


## Gridsearchcv?

In [42]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':range(3,20)}
clf = GridSearchCV(DecisionTreeClassifier(), parameters, n_jobs=4)
clf.fit(X_train, y_train)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

0.7481481481481481 {'max_depth': 19}


In [44]:
clf.score(X_test,y_test)

0.7593434343434343

In [45]:
df_test_labels3 = clf.predict(df_test_values)

In [46]:
df_sub['status_group'] = df_test_labels3

In [47]:
df_sub['status_group'] = df_sub['status_group'].astype(str)

for i in range(len(df_sub.status_group)):
    if df_sub.status_group[i] == '3':
        df_sub.at[i, 'status_group'] = 'functional'
    elif df_sub.status_group[i] == '2':
        df_sub.at[i, 'status_group'] = 'functional needs repair'
    else:
        df_sub.at[i, 'status_group'] = 'non functional'
df_sub.sample(3)

,id,status_group
4544,65748,functional
1462,56896,non functional
9760,54331,functional


In [48]:
df_sub.status_group.value_counts()

functional                 8548
non functional             4428
functional needs repair    1874
Name: status_group, dtype: int64

In [49]:
df_sub.to_csv('./Submissions/decisionTree5_8_2pm.csv',index=False)

## Ranking

<table>
  <tr>
    <th>Best</th>
    <th>Current Rank</th>
    <th>Competitors</th>
    <th>Time</th>
  </tr>
  <tr>
    <td>0.6527</td>
    <td>Not Best</td>
    <td>1277</td>
    <td>May 8, 2018, 9:04 p.m.</td>
  </tr>
</table>

## Random forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
clf = RandomForestClassifier(max_depth = 4,random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [34]:
print(clf.feature_importances_)

[2.93795595e-02 5.89454173e-03 1.98662616e-02 4.86895152e-03
 0.00000000e+00 3.15100067e-03 1.61185660e-03 1.04629073e-03
 7.95584103e-05 7.70558867e-04 5.13397963e-03 4.29024272e-04
 0.00000000e+00 2.22169935e-02 1.68787983e-02 6.57595198e-03
 3.57331111e-02 1.45383383e-04 2.73851949e-03 0.00000000e+00
 4.13003337e-02 6.94984854e-05 3.30483445e-03 2.50998123e-02
 0.00000000e+00 3.56657123e-06 0.00000000e+00 3.07938771e-01
 1.39937799e-01 5.67622310e-04 5.05061876e-03 2.14568017e-03
 6.81474747e-05 1.13772297e-03 0.00000000e+00 0.00000000e+00
 1.14925042e-02 0.00000000e+00 6.88810831e-03 6.02363293e-02
 3.59157350e-03 1.50798850e-03 0.00000000e+00 0.00000000e+00
 9.27505190e-03 0.00000000e+00 6.53611821e-03 5.90272604e-03
 1.87034217e-01 3.49291035e-03 1.47500555e-04 9.05279686e-06
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.25120350e-03 5.57978629e-04 2.92545986e-03 4.12947385e-04
 2.62478229e-03 1.29687981e-02 0.00000000e+00]


In [53]:
df_test_labels2 = clf.predict(df_test_values)

In [54]:
df_sub['status_group'] = df_test_labels2

In [55]:
df_sub['status_group'] = df_sub['status_group'].astype(str)

for i in range(len(df_sub.status_group)):
    if df_sub.status_group[i] == '3':
        df_sub.at[i, 'status_group'] = 'functional'
    elif df_sub.status_group[i] == '2':
        df_sub.at[i, 'status_group'] = 'functional needs repair'
    else:
        df_sub.at[i, 'status_group'] = 'non functional'
df_sub.sample(3)

,id,status_group
12303,12395,functional
212,3782,functional
14581,8930,functional


In [56]:
df_sub.status_group.value_counts()

functional        11901
non functional     2949
Name: status_group, dtype: int64

In [57]:
#df_sub.to_csv('./Submissions/decisionTree2.csv',index=False)

## Ranking

<table>
  <tr>
    <th>Best</th>
    <th>Current Rank</th>
    <th>Competitors</th>
    <th>Time</th>
  </tr>
  <tr>
    <td>0.6988</td>
    <td>1276</td>
    <td>5059</td>
    <td>May 8, 2018, 12:13 a.m.</td>
  </tr>
</table>

## Gridsearchcv?

In [43]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':range(3,20)}
clf = GridSearchCV(RandomForestClassifier(), parameters, n_jobs=4)
clf.fit(X_train, y_train)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

0.7786531986531987 {'max_depth': 19}


In [120]:
clf.score(X_test,y_test)

0.7859427609427609

In [44]:
df_test_labels3 = clf.predict(df_test_values)

In [46]:
prob = clf.predict_proba(df_test_values)
prob

array([[5.00000000e-01, 1.00000000e-01, 4.00000000e-01],
       [2.74530974e-01, 5.72254485e-02, 6.68243578e-01],
       [5.00000000e-01, 1.00000000e-01, 4.00000000e-01],
       ...,
       [8.73853319e-02, 1.07532319e-01, 8.05082349e-01],
       [2.29487179e-01, 8.33333333e-03, 7.62179487e-01],
       [9.94149898e-01, 1.21506683e-04, 5.72859556e-03]])

In [47]:
prob[0].max()

0.5

In [48]:
df_sub['status_group'] = df_test_labels3

In [49]:
df_sub.sample(5)

,id,status_group
3680,63260,3
6162,48626,3
4275,4815,3
4199,54001,3
2747,26700,1


In [50]:
how = []
for i in range(len(prob)):
    if prob[i].max() <= 0.54:
        df_sub.at[i,'status_group'] = 3
df_sub.sample(10)

,id,status_group
11885,23669,3
13827,39037,3
9993,69856,3
11845,16647,1
8569,31637,3
6760,20021,2
12030,29178,3
5152,53553,3
94,57508,3
13343,54310,3


In [51]:
df_sub['status_group'] = df_sub['status_group'].astype(str)

for i in range(len(df_sub.status_group)):
    if df_sub.status_group[i] == '3':
        df_sub.at[i, 'status_group'] = 'functional'
    elif df_sub.status_group[i] == '2':
        df_sub.at[i, 'status_group'] = 'functional needs repair'
    else:
        df_sub.at[i, 'status_group'] = 'non functional'
df_sub.sample(3)

,id,status_group
3829,20660,functional
7467,72539,functional
955,55439,functional


In [52]:
df_sub.status_group.value_counts()

functional                 10929
non functional              3797
functional needs repair      124
Name: status_group, dtype: int64

In [53]:
df_sub.to_csv('./Submissions/decisionTree5_8_730pm.csv',index=False)

# Appendix

## A

In [ ]:
columns = list(feature_sel.columns)
feature_sel[columns[5:]] = feature_sel[columns[5:]].astype(int)
feature_sel[columns[0]] = feature_sel[columns[0]].astype(int)

In [ ]:
feature_sel = feature_sel.sample(frac=1).reset_index(drop=True)
y = feature_sel.status_group
X = feature_sel.drop('status_group', axis=1)

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.84)

In [ ]:
X_train.head(1)

## Adjust competition data

In [ ]:
df_test_values = df_test_values[['gps_height', 'longitude', 'latitude', 'basin',
                  'extraction_type_class','payment','quantity','water_quality', 'source_class',
                  'waterpoint_type_group','population','public_meeting','scheme_management']]

In [ ]:
# rearange columns
cols = df_test_values.columns.tolist()
cols = ['gps_height',
             'longitude',
             'latitude',
             'population',
             'basin',
             'extraction_type_class',
             'payment',
             'quantity',
             'water_quality',
             'source_class',
             'waterpoint_type_group',
             'public_meeting',
             'scheme_management']
df_test_values = df_test_values[cols]

In [ ]:
a = [0] * len(df_test_values['gps_height'])

In [ ]:
df_test_values = pd.get_dummies(df_test_values,columns=list(df_test_values.columns[4:]))
df_test_values.sample(5)

In [ ]:
columns = list(feature_sel.columns)
feature_sel[columns[5:]] = feature_sel[columns[5:]].astype(int)
feature_sel[columns[0]] = feature_sel[columns[0]].astype(int)

In [ ]:
df_test_values['scheme_management_None'] = a
df_test_values.head(10)

In [ ]:
list(set(X_train.columns)-set(df_test_values.columns))

##  B

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd

from IPython.display import Image

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn import svm

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
pd.Series(y_train).value_counts()

## Principle Component Analysis

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)

In [ ]:
pcafeatures_train = pca.transform(X_train)

In [ ]:
from itertools import cycle

In [ ]:
# def plot_PCA_2D(data, target, target_names):
#     colors = cycle(['r','g','b'])
#     target_ids = range(len(target_names))
#     plt.figure()
#     for i, c, label in zip(target_ids, colors, target_names):
#         plt.scatter(data[target == i, 0], data[target == i, 1],
#                    c=c, label=label)
#     plt.legend()

In [ ]:
# plot_PCA_2D(pcafeatures_train, target=y_train, target_names=digits.target_names)

## Fitting Linear and RBF SVM Models

In [ ]:
# fit linear model
model_svm = svm.SVC(kernel='rbf',probability=False,cache_size=2000)
model_svm.fit(X_train, y_train)

In [ ]:
# predict out of sample
y_pred = model_svm.predict(X_test)

In [ ]:
# check accuracy
accuracy_score(y_test,y_pred)

In [ ]:
# confusion matrix
confusion_matrix(y_test,y_pred)

In [ ]:
# fit rbf model
# model_svm2 = svm.SVC(kernel='rbf', gamma = 0.001)
# model_svm2.fit(X_train, y_train)

In [ ]:
# predict out of sample
y_pred2 = model_svm2.predict(X_test)

In [ ]:
# check accuracy
accuracy_score(y_test,y_pred2)

In [ ]:
# confusion matrix
confusion_matrix(y_test,y_pred2)

## C

Extra code from class to utilize

In [ ]:
df.age=df.age.fillna(df.age.mean())

In [ ]:
y,X=dmatrices('survived~ pclass +age+sibsp+parch+fare',data=df,return_type='dataframe')

In [ ]:
# Generate a confusion matrix plot: 

def plot_confusion_matrix(cm,title='Confusion matrix', cmap=plt.cm.Reds):
    plt.imshow(cm, interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#Could be a typical function for classifying:

def train_score(classifier,x,y):
    xtrain, xtest, ytrain, ytest = cross_validation.train_test_split(x, y, test_size=0.2, random_state=1234)
    ytrain=np.ravel(ytrain)
    clf = classifier.fit(xtrain, ytrain)
    # accuracy for test & train:
    train_acc=clf.score(xtrain, ytrain)
    test_acc=clf.score(xtest,ytest)
    print("Training Data Accuracy: %0.2f" %(train_acc))
    print("Test Data Accuracy:     %0.2f" %(test_acc))
    
    y_true = ytest
    y_pred = clf.predict(xtest)


    conf = confusion_matrix(y_true, y_pred)
    print(conf)

    print ('\n')
    print ("Precision:              %0.2f" %(conf[0, 0] / (conf[0, 0] + conf[1, 0])))
    print ("Recall:                 %0.2f"% (conf[0, 0] / (conf[0, 0] + conf[0, 1])))
    
    cm=confusion_matrix(y_true, y_pred, labels=None)
    
    plt.figure()
    plot_confusion_matrix(cm)

In [ ]:
log_clf=LogisticRegression()
train_score(log_clf,X,y)

In [ ]:
# What about ROC ? 

from sklearn.metrics import roc_curve, auc

xtrain, xtest, ytrain, ytest = cross_validation.train_test_split(X, y, test_size=0.2, random_state=1234)
log = LogisticRegression()
log.fit(xtrain,np.ravel(ytrain))
y_score=log.predict_proba(xtest)[:,1]

fpr, tpr,_ = roc_curve(ytest, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
# Plotting our Baseline..
plt.plot([0,1],[0,1])
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')

In [ ]:
tpr

####  Cost Benefit Example: 

We can also optimize our models based on specific costs associated with our classification errors; here we will use specific dollar amounts as weights.

Let's say we were developing a classification model for Aircraft Delay prediction.  For this example let's assume that a true positive would 
lead to a cost savings of 2160 dollars, a false negative would cost us 2900 dollars a false positive would cost 750 dollars.  

cb = np.array([[2160, -750.0], [-2900, 0]])  

Expected_Value = #TPs(2160) - #FNs(2900) -#FPs(750)  